In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
account = pd.read_csv('account.csv',sep=';')
card = pd.read_csv('card.csv', sep=';')
trans = pd.read_csv('trans.csv', sep=';')
client = pd.read_csv('client.csv', sep = ';')
order = pd.read_csv('order.csv', sep = ';')
loan = pd.read_csv('loan.csv', sep = ';')
disp = pd.read_csv('disp.csv', sep = ';')
district = pd.read_csv('district.csv', sep = ';')

last_date = '981231'

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [126]:
insurance = trans['k_symbol'] == "POJISTNE"
ins_holder = set(trans[insurance]['account_id'])
26 in ins_holder

False

In [132]:
accid = account['account_id'].values
card = disp[disp['type'] == 'OWNER']['account_id'].values
insurance = trans['k_symbol'] == "POJISTNE"
ins_holder = set(trans[insurance]['account_id'])
customer = []
for x in accid:
    bal = trans[trans['account_id'] == x].sort_values('date').iloc[[-1]]['balance'].values[0]
    lastxtn = trans[trans['account_id'] == x].sort_values('date').iloc[[-1]]['date'].values[0]
    loanstatus = loan[['account_id', 'status']].values
    
    # Account, balance and HasCard status
    if x in card:
        customer.append([x,bal,1])
    else:
        customer.append([x,bal,0])
    
    xxlen = len(customer) - 1    
    
    # Loan status of account holders
    if x in loanstatus:
        customer[xxlen] = customer[xxlen] + [loanstatus[0][1]]
    else:
        customer[xxlen] = customer[xxlen] + [np.nan]
        
    # Insurance holders
    if x in ins_holder:
        customer[xxlen] = customer[xxlen] + [1]
    else:
        customer[xxlen] = customer[xxlen] + [0]
    # Check for dormancy
    if lastxtn <= 981031:
        customer[xxlen] = customer[xxlen] + [1]
    else:
        customer[xxlen] = customer[xxlen] + [0]  
    

In [133]:
customer

[[576, 35550.7, 1, nan, 0, 0],
 [3818, 62199.8, 1, nan, 1, 0],
 [704, 81680.3, 1, nan, 0, 0],
 [2378, 93349.5, 1, nan, 0, 0],
 [2632, 62959.3, 1, nan, 1, 0],
 [1972, 23740.2, 1, nan, 0, 0],
 [1539, 8957.3, 1, nan, 0, 0],
 [793, 19396.2, 1, nan, 0, 0],
 [2484, 43644.0, 1, nan, 1, 0],
 [1695, 106897.4, 1, nan, 0, 0],
 [1726, 34243.8, 1, nan, 1, 0],
 [2881, 52610.8, 1, nan, 0, 0],
 [2357, 38296.6, 1, nan, 0, 0],
 [2177, 32458.4, 1, nan, 0, 0],
 [485, 79053.2, 1, nan, 1, 0],
 [652, 68449.9, 1, nan, 0, 0],
 [9635, 6921.1, 1, nan, 0, 0],
 [1844, 43524.8, 1, nan, 0, 0],
 [1926, 22197.6, 1, nan, 0, 0],
 [2393, 48060.5, 1, nan, 0, 0],
 [3510, 47529.7, 1, nan, 0, 0],
 [1276, 22799.9, 1, nan, 1, 0],
 [1637, 34367.0, 1, nan, 0, 0],
 [3871, 59739.6, 1, nan, 0, 0],
 [192, 29946.5, 1, nan, 0, 0],
 [764, 21491.3, 1, nan, 0, 0],
 [2087, 61954.5, 1, nan, 0, 0],
 [2122, 41803.8, 1, nan, 0, 0],
 [3592, 21075.9, 1, nan, 0, 0],
 [3487, 80704.8, 1, nan, 0, 0],
 [435, 27135.0, 1, nan, 0, 0],
 [1730, 67162.7, 